# Benchmarking fairness on test DaNE dataset
Working progress

**Disclaimer:** The name dataset with ethnicities is from 2010 and only has female names with other ehtnicities than Danish.


In [32]:
#!pip install git+https://github.com/KennethEnevoldsen/DaCy
!pip install conllu
!pip install tabulate
!pip install seqeval
!pip install xlrd
!pip install openpyxl
!pip install transformers==3.6.0

Could not fetch URL https://pypi.org/simple/transformers/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/transformers/ (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available.")) - skipping
ERROR: Could not find a version that satisfies the requirement transformers==3.5.1
ERROR: No matching distribution found for transformers==3.5.1


# Utils

In [23]:
from tabulate import tabulate
import numpy as np
from seqeval.metrics import classification_report, f1_score


def f1_class(k, true, pred):
    tp = np.sum(np.logical_and(pred == k, true == k))

    fp = np.sum(np.logical_and(pred == k, true != k))
    fn = np.sum(np.logical_and(pred != k, true == k))
    if tp == 0:
        return 0
    recall = tp / (tp + fp)
    precision = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    return precision, recall, f1


def f1_report(true, pred, modelname="", dataname="", word_level=False, bio=False):

    if bio:
        return classification_report(true, pred, digits=4)

    if word_level:
        true = [tag for sent in true for tag in sent]
        pred = [tag for sent in pred for tag in sent]

    data_b = []
    data_a = []
    headers_b = ["{} // {} ".format(modelname, dataname), 'Class', 'Precision', 'Recall', 'F1', 'support']
    headers_a = ['Accuracy', 'Avg-f1', 'Weighted-f1', '', '']
    aligns_b = ['left', 'left', 'center', 'center', 'center']

    true = np.array(true)
    pred = np.array(pred)
    acc = np.sum(true == pred) / len(true)

    n = len(np.unique(true))
    avg = 0
    wei = 0
    for c in np.unique(true):
        precision, recall, f1 = f1_class(c, pred, true)
        avg += f1 / n
        wei += f1 * (np.sum(true == c) / len(true))

        data_b.append(['', c, round(precision, 4), round(recall, 4), round(f1, 4)])
    data_b.append(['', '', '', '', ''])
    data_b.append(headers_a)
    data_b.append([round(acc, 4), round(avg, 4), round(wei, 4), '', ''])
    print()
    print(tabulate(data_b, headers=headers_b, colalign=aligns_b), '\n')
    return data_b

# Load data

In [24]:
from danlp.datasets import DDT
import pandas as pd
import copy

In [25]:
def is_misc(ent: str):
    if len(ent) < 4:
        return False
    return ent[-4:] == 'MISC'

def isolate_per(se: list):
    return [
        [entity if entity in ['B-PER', 'I-PER'] else 'O' for entity in entities]
        for entities in se
    ]

def remove_miscs(se: list):
    return [
        [entity if not is_misc(entity) else 'O' for entity in entities]
        for entities in se
    ]

# Load the DaNE data
_, _, test = DDT().load_as_simple_ner(predefined_splits=True)
sentences_tokens, sentences_entities = test

# Replace MISC with O for fair comparisons
sentences_entities = remove_miscs(sentences_entities)

In [26]:
num_sentences = len(sentences_tokens)
num_tokens = sum([len(s) for s in sentences_tokens])
print(num_sentences, num_tokens)

565 10023


# Mutate to firstn name to new ethnicity
Issue here is that all other ethnicities' names are only female

In [27]:
danish_accepted_names_2010 = pd.read_excel('data/Godkendte Danske Fornavne.xlsx')
print(danish_accepted_names_2010['Etnicity'].unique())
print(danish_accepted_names_2010.keys())
print(len(danish_accepted_names_2010))
# remove firstnames with spaces (middle names)
cols = [ ' ' not in c for c in danish_accepted_names_2010['FirstName'] ]
danish_accepted_names_2010 = danish_accepted_names_2010[cols] 
print(len(danish_accepted_names_2010))

['Dansk' 'muslimsk' 'tamilsk' 'afrikansk' 'burma' nan]
Index(['FirstName', 'Sex', 'Etnicity'], dtype='object')
24560
24339


In [28]:
# Get all other ethnicity than danish first names
other_names = danish_accepted_names_2010[danish_accepted_names_2010['Etnicity'] != 'Dansk']['FirstName']
print(other_names.sample().item())

danish_names = danish_accepted_names_2010[danish_accepted_names_2010['Etnicity'] == 'Dansk'][danish_accepted_names_2010['Sex'] == 'Female']['FirstName']
print(danish_names.sample().item())

Nujaym
Wiwi
<ipython-input-28-835393a60621>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  danish_names = danish_accepted_names_2010[danish_accepted_names_2010['Etnicity'] == 'Dansk'][danish_accepted_names_2010['Sex'] == 'Female']['FirstName']


In [29]:
def get_nondanish_dataset(tokens):
    sentences_tokens_other = copy.deepcopy(tokens)
    for i in range(len(sentences_tokens_other)):
        for j in range(len(sentences_tokens_other[i])):
            if sentences_entities[i][j] == 'B-PER':
                sentences_tokens_other[i][j] = other_names.sample().item()
    return sentences_tokens_other

In [30]:
def get_randomized_danish_dataset(tokens):
    sentences_tokens_danish = copy.deepcopy(tokens)
    for i in range(len(sentences_tokens_danish)):
        for j in range(len(sentences_tokens_danish[i])):
            if sentences_entities[i][j] == 'B-PER':
                sentences_tokens_danish[i][j] = danish_names.sample().item()

    return sentences_tokens_danish

# Test DaNLP BERT NER model original DaNE test set

In [31]:
from danlp.models import load_bert_ner_model
bert = load_bert_ner_model()

predictions = []
for i, sentence in enumerate(sentences_tokens):
    _, pred_ents = bert.predict(sentence)
    predictions.append(pred_ents)
print('BERT:')

assert len(predictions) == num_sentences
data = f1_report(sentences_entities, remove_miscs(predictions), bio=True)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
f1_score(isolate_per(sentences_entities), isolate_per(predictions))

## Sanity check (mutating danish to danish female names)


In [ ]:
from danlp.models import load_bert_ner_model
bert = load_bert_ner_model()

# Bootstrap results
F1_PER = list()
for j in range(100):
    predictions = []
    sentences_tokens_danish = get_randomized_danish_dataset(sentences_tokens)
    for i, sentence in enumerate(sentences_tokens_danish):
        _, pred_ents = bert.predict(sentence)
        predictions.append(pred_ents)

    F1_PER.append(f1_score(isolate_per(sentences_entities), isolate_per(predictions)))
    print(j, F1_PER[-1])
result = np.array(F1_PER)
print(result.mean(), result.std())

0 0.9
1 0.9035812672176309
2 0.8997289972899728
3 0.9293478260869564
4 0.9150684931506851
5 0.9159891598915989
6 0.9234972677595629
7 0.8937329700272479
8 0.8991825613079019
9 0.9010989010989011
10 0.888283378746594
11 0.9056603773584906
12 0.9209809264305178
13 0.9272237196765499
14 0.905149051490515
15 0.9016393442622951
16 0.9405405405405406
17 0.862533692722372
18 0.89196675900277
19 0.9095890410958904
20 0.9293478260869564
21 0.9071038251366121
22 0.907608695652174
23 0.9110512129380054
24 0.9100817438692099
25 0.9130434782608695
26 0.8972972972972973
27 0.9021739130434783
28 0.9139784946236558
29 0.8907103825136612
30 0.9130434782608695
31 0.8888888888888888
32 0.8975069252077562
33 0.9046321525885559
34 0.9130434782608695
35 0.9226519337016573
36 0.8980716253443526
37 0.9046321525885559
38 0.8888888888888887
39 0.8937329700272479
40 0.8876712328767123
41 0.9175824175824175
42 0.9002695417789757
43 0.8997289972899728
44 0.9180327868852459
45 0.8956043956043955
46 0.88888888888888

# Test DaNLP BERT NER model mutated other ethnicity first name DaNE test set

In [ ]:
# Bootstrap results
F1_PER = list()
for j in range(100):
    predictions = []
    sentences_tokens_other = get_nondanish_dataset(sentences_tokens)
    for i, sentence in enumerate(sentences_tokens_other):
        _, pred_ents = bert.predict(sentence)
        predictions.append(pred_ents)

    F1_PER.append(f1_score(isolate_per(sentences_entities), isolate_per(predictions)))
    print(j, F1_PER[-1])
result = np.array(F1_PER)
print(result.mean(), result.std())

0 0.8876712328767123
1 0.8594594594594595
2 0.905149051490515
3 0.9010989010989011
4 0.8815426997245178
5 0.9065934065934065
6 0.8858695652173912
7 0.8743169398907102
8 0.8956043956043955
9 0.8797814207650273
10 0.8876712328767123
11 0.88283378746594
12 0.8695652173913044
13 0.8719346049046323
14 0.8937329700272479
15 0.8991825613079019
16 0.891304347826087
17 0.8950276243093923
18 0.8858695652173912
19 0.89196675900277
20 0.888283378746594
21 0.894308943089431
22 0.8870523415977961
23 0.8743169398907102
24 0.858695652173913
25 0.905149051490515
26 0.8907103825136612
27 0.8702702702702702
28 0.883977900552486
29 0.858695652173913
30 0.883977900552486
31 0.9071038251366121
32 0.8918918918918918
33 0.9105691056910569
34 0.8698060941828255
35 0.8876712328767123
36 0.8648648648648649
37 0.8907103825136612
38 0.89247311827957
39 0.8918918918918918
40 0.8780487804878048
41 0.9120879120879122
42 0.8617886178861788
43 0.9032258064516129
44 0.8797814207650273
45 0.8834688346883468
46 0.86813186

# Prepare data for DaCy

In [33]:
import os
import pyconll
from danlp.download import DEFAULT_CACHE_DIR, download_dataset, _unzip_process_func, DATASETS
dataset_dir = download_dataset('ddt', process_func=_unzip_process_func, cache_dir=DEFAULT_CACHE_DIR)

## Modified data class 

In [34]:
class DDT_:
    """
    Class for loading the Danish Dependency Treebank (DDT) through several frameworks/formats.   
    The DDT dataset has been annotated with NER tags in the IOB2 format.
    The dataset is downloaded in CoNLL-U format, but with this class
    it can be converted to spaCy format or a simple NER format
    similar to the CoNLL 2003 NER format.
    :param str cache_dir: the directory for storing cached models
    :param bool verbose: `True` to increase verbosity
    """
    def __init__(self, cache_dir: str = DEFAULT_CACHE_DIR):
        self.dataset_name = 'ddt'
        self.file_extension = DATASETS[self.dataset_name]['file_extension']
        self.dataset_dir = download_dataset('ddt', process_func=_unzip_process_func, cache_dir=cache_dir)

    def load_as_conllu(self, predefined_splits: bool = False):
        """
        Load the DDT in CoNLL-U format.Œ
        :param bool predefined_splits:
        :return: A single pyconll.Conll
                or a tuple of (train, dev, test) pyconll.Conll
                depending on predefined_split
        """

        parts = [None, None, None]  # Placeholder list to put predefined parts of dataset [train, dev, test]
        for i, part in enumerate(['train', 'dev', 'test']):
            file_name = "{}.{}{}".format(self.dataset_name, part, self.file_extension)
            file_path = os.path.join(self.dataset_dir, file_name)

            parts[i] = pyconll.load_from_file(file_path)

        # if predefined_splits: then we should return three files
        if predefined_splits:
            return parts

        # Merge the splits to one single dataset
        parts[0].extend(parts[1])
        parts[0].extend(parts[2])

        return parts[0]

    def load_as_simple_ner(self, predefined_splits: bool = False):
        conllu_parts = self.load_as_conllu(predefined_splits)

        if not predefined_splits:
            conllu_parts = [conllu_parts]

        parts = []
        for conllu_part in conllu_parts:
            full_sentences = []
            part_sentences = []
            part_entities = []

            for sent in conllu_part:
                part_sentences.append([token.form for token in sent._tokens])
                part_entities.append([token.misc['name'].pop() for token in sent._tokens])
                full_sentences.append(sent)

            parts.append([part_sentences, part_entities, full_sentences])

        if predefined_splits:
            return parts
        return parts[0]


    def load_with_flair(self, predefined_splits: bool = False):
        """
        Load the DDT with flair. 
        This function is inspired by the "Reading Your Own Sequence Labeling Dataset" from Flairs tutorial
        on reading corpora:
        https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md
        :param predefined_splits:
        :type predefined_splits: bool
        :return: ColumnCorpus
        .. note:: TODO: Make a pull request to flair similar to this:
            https://github.com/zalandoresearch/flair/issues/383
        """

        from flair.data import Corpus
        from flair.datasets import ColumnCorpus

        columns = {1: 'text', 3: 'pos', 9: 'ner'}

        # init a corpus using column format, data folder and the names of the train, dev and test files
        corpus: Corpus = ColumnCorpus(self.dataset_dir, columns, comment_symbol='#',
                                      train_file='{}.{}{}'.format(self.dataset_name, 'train', self.file_extension),
                                      test_file='{}.{}{}'.format(self.dataset_name, 'test', self.file_extension),
                                      dev_file='{}.{}{}'.format(self.dataset_name, 'dev', self.file_extension))

        # Remove the `name=` from `name=B-PER` to only use the `B-PER` tag
        parts = ['train', 'dev', 'test']
        for part in parts:
            dataset = corpus.__getattribute__(part)

            for sentence in dataset.sentences:
                for token in sentence.tokens:
                    if 'ner' in token.tags:
                        token.tags['ner'].value = token.tags['ner'].value.split("=")[1].replace("|SpaceAfter", "")

        return corpus

    def load_with_spacy(self):
        """
        Loads the DDT with spaCy. 
        
        This function converts the conllu files to json in the spaCy format.
        :return: GoldCorpus
        .. note:: Not using jsonl because of:
            https://github.com/explosion/spaCy/issues/3523
        """
        import srsly
        from spacy.cli.converters import conllu2json
        from spacy.gold import GoldCorpus
        from spacy.gold import Path

        for part in ['train', 'dev', 'test']:
            conll_path = os.path.join(self.dataset_dir, '{}.{}{}'.format(self.dataset_name, part, self.file_extension))
            json_path = os.path.join(self.dataset_dir, "ddt.{}.json".format(part))

            if not os.path.isfile(json_path):  # Convert the conllu files to json
                with open(conll_path, 'r') as file:
                    file_as_string = file.read()
                    file_as_string = file_as_string.replace("name=", "").replace("|SpaceAfter=No", "")
                    file_as_json = conllu2json(file_as_string)

                    srsly.write_json(json_path, file_as_json)

        train_json_path = os.path.join(self.dataset_dir, "ddt.train.json")
        dev_json_path = os.path.join(self.dataset_dir, "ddt.dev.json")

        assert os.path.isfile(train_json_path)
        assert os.path.isfile(dev_json_path)

        return GoldCorpus(Path(train_json_path), Path(dev_json_path))

## Load data with modified class

In [35]:

# Load the DaNE data
_, _, test = DDT_().load_as_simple_ner(predefined_splits=True)
sentences_tokens, sentences_entities, sentences = test

# Replace MISC with O for fair comparisons
sentences_entities = remove_miscs(sentences_entities)
sentences[0].text

'To kendte russiske historikere Andronik Mirganjan og Igor Klamkin tror ikke, at Rusland kan udvikles uden en "jernnæve".'

# Test DaCy original data + sanity check danish dataset 

In [38]:
import dacy
import spacy
spacy.prefer_gpu()
ner_model = dacy.load("da_dacy_large_tft-0.0.0")

0da7cb975b245d9e6574458c7c89dfd9?download: 1.82GB [11:26, 2.65MB/s]                            


In [39]:
ner_model.pipeline
from spacy.tokens import Doc
doctest = Doc(ner_model.vocab, words=sentences_tokens[0])
doctest_out = doctest
for pipe in ner_model.pipeline:
    doctest_out = pipe[1](doctest_out)


In [42]:
def isolate_per_(se: list):
    return [
        ['PER' if entity in ['B-PER', 'I-PER'] else 'O' for entity in entities]
        for entities in se
    ]
predictions = []

for i, words in enumerate(sentences_tokens):
    doc = Doc(ner_model.vocab, words=words)
    for pipe in ner_model.pipeline:
        doc = pipe[1](doc)
    pred = [ x.ent_type_ if 'PER' in x.ent_type_  else 'O' for x in doc ]
    if len(sentences_entities[i]) == len(pred):        
        predictions.append(pred)
    else:
        print(sentences_entities[i], pred, sentences[i].text) 
    
print(f1_score(isolate_per_(sentences_entities), predictions))

0.9415041782729804
/Users/au561649/.virtualenvs/NLP/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [45]:
len(sentences_entities)

565

In [46]:
len(predictions)

565

In [49]:
for idx, i in enumerate(zip(sentences_entities, predictions)):
    if len(i[0]) != len(i[1]):
        print(idx)

In [60]:
wrong_pred = []
for idx, i in enumerate(zip(isolate_per_(sentences_entities), predictions)):
    for y, y_hat in zip(i[0], i[1]):
        if y != y_hat:
            print(idx)
            wrong_pred.append(idx)
            break
        

72
158
237
241
253
259
260
307
313
316
319
320
387
418
423
475


In [64]:
for idx in wrong_pred:
    print("----")
    print(" ".join(sentences_tokens[idx]))
    print(list(zip(predictions[idx], sentences_entities[idx])))

----
Her morer Super-Pedersen sig over Nettos priser
[('O', 'O'), ('O', 'O'), ('O', 'B-PER'), ('O', 'O'), ('O', 'O'), ('O', 'B-ORG'), ('O', 'O')]
----
Psykedelisk-mønstret klokkehat 149 kr. og Marilyn Monroe badedragt i kulørt Lycra med Marilyn fotografier på numsen 299 kr. - som også er prisen for mange af husets andre badedragter .
[('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('PER', 'B-PER'), ('PER', 'I-PER'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('PER', 'B-PER'), ('PER', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O')]
----
Alcala havde punktering efter fem kilometer , og Greg Lemond punkterede efter kun 10 meters kørsel .
[('O', 'B-PER'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('PER', 'B-PER'), ('PER', 'I-PER'), ('O', 'O'), ('O', 'O'), ('O', 'O'), ('O'

In [72]:
from spacy import displacy
doc = ner_model("""Depeche har det selv morsomt mens de indspiller pladen , hvor Martin f.eks. går på " Jungle Club " og ser på mandestrip , men bortset fra en samplet optagelse af Dave der drejer tændingsnøglen i sin nyindkøbte Porsche , er der ikke meget at grine af på " Black Celebration" """)
displacy.render(doc, style="ent")

## Sanity check (mutating danish to other ethnicity female names)


In [41]:
# Bootstrap results
F1_PER = list()
for j in range(100):
    predictions = []
    sentences_tokens_other = get_nondanish_dataset(sentences_tokens)
    for i, sentence in enumerate(sentences_tokens_other):
        doc = Doc(ner_model.vocab, words=sentence)
        for pipe in ner_model.pipeline:
            doc = pipe[1](doc)
        pred = [ x.ent_type_ if 'PER' in x.ent_type_  else 'O' for x in doc ]
        predictions.append(pred)

    F1_PER.append(f1_score(isolate_per_(sentences_entities), predictions))
    print(j, F1_PER[-1])
result = np.array(F1_PER)
print(result.mean(), result.std())

0 0.9352112676056339
1 0.9415041782729804


KeyboardInterrupt: 

## Sanity check (mutating danish to danish female names)


In [ ]:
# Bootstrap results
F1_PER = list()
for j in range(100):
    predictions = []
    sentences_tokens_danish = get_randomized_danish_dataset(sentences_tokens)
    for i, sentence in enumerate(sentences_tokens_danish):
        doc = Doc(ner_model.vocab, words=sentence)
        for pipe in ner_model.pipeline:
            doc = pipe[1](doc)
        pred = [ x.ent_type_ if 'PER' in x.ent_type_  else 'O' for x in doc ]
        predictions.append(pred)

    F1_PER.append(f1_score(isolate_per_(sentences_entities), predictions))
    print(j, F1_PER[-1])
result = np.array(F1_PER)
print(result.mean(), result.std())

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


0 0.9444444444444444
1 0.9589041095890412
2 0.9529085872576178
3 0.9444444444444444
4 0.9441340782122906
5 0.9415041782729804
6 0.9502762430939227
7 0.9441340782122906
8 0.9473684210526315
9 0.9415041782729804
10 0.956043956043956
11 0.9382022471910112
12 0.953168044077135
13 0.9295774647887323
14 0.9500000000000001
15 0.9500000000000001
16 0.9441340782122906
17 0.9411764705882353
18 0.9325842696629215
19 0.9418282548476455
20 0.9444444444444444
21 0.9502762430939227
22 0.9502762430939227
23 0.9355742296918768
24 0.9388888888888889
25 0.9385474860335196
26 0.9441340782122906
27 0.9329608938547486
28 0.9291784702549574
29 0.9299719887955182
30 0.9500000000000001
31 0.9352112676056339
32 0.9415041782729804
33 0.9444444444444444
34 0.9500000000000001
35 0.9385474860335196
36 0.935933147632312
37 0.947075208913649
38 0.9473684210526315
39 0.9529085872576178
40 0.9558011049723757
41 0.9473684210526315
42 0.9269662921348315
43 0.9411764705882353
44 0.9586776859504132
45 0.9500000000000001
46